In [8]:
import re #pacchetto per espressioni regolari
import os #pacchetto per muoversi nelle cartelle
import sys
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *


In [9]:
train_path = 'data/training.txt'
test_path = 'data/test.txt'
out_dir = 'data/profiling_input'

In [3]:
# Necessario etichettare il blind test

# Read gold.txt file and extract gender values
with open("data/gold.txt", "r") as gold_file:
    gold_data = gold_file.readlines()

# Create a dictionary to store the gender values
gender_dict = {}
for line in gold_data:
    line_parts = line.strip().split("\t")
    doc_id = line_parts[0]
    gender = line_parts[1]
    gender_dict[doc_id] = gender

# Read test.txt file and replace "?" with gender values
with open("data/test.txt", "r") as test_file:
    test_data = test_file.readlines()

# Replace "?" with gender values from gold.txt
updated_test_data = []
for line in test_data:
    if "?" in line:
        doc_ids = re.findall(r'id="(.*?)"', line)
        if doc_ids:  # Check if any doc_ids were found
            doc_id = doc_ids[0]  # Take the first doc_id found
            gender = gender_dict.get(doc_id, "?")
            updated_line = line.replace("?", gender)
            updated_test_data.append(updated_line)
        else:
            updated_test_data.append(line)  # If no doc_id found, append line unchanged
    else:
        updated_test_data.append(line)

# Write the updated test data to a new file
with open("data/test.txt", "w") as updated_test_file:
    updated_test_file.writelines(updated_test_data)

In [10]:
for file_path in [train_path, test_path]:
    split = get_split(file_path)

    for line in open(file_path, 'r'):
        line = line.strip() # rimuoviamo eventuali spazi e ritorni a capo

        if '<doc id="' in line:    # troviamo un nuovo documento
            doc_id, genre, gender = get_doc_info(line)
            out_file_name = f'{split}#{doc_id}#{genre}#{gender}.txt'
            out_file_path = os.path.join(out_dir, out_file_name)
            user_post = []

        elif '</doc>' in line:     # fine dell'utente
            with open(out_file_path, 'w', encoding="utf-8") as out_file:
                for post in user_post:
                    out_file.write(post)
                    out_file.write('\n')

        elif line != '':  # siamo in corrispondenza del testo di un post
            user_post.append(line)

In [22]:
import os
import chardet

# Directory contenente i file da convertire
directory = 'data/profiling_input'

# Ciclo attraverso ogni file nella directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Rileva la codifica del file
        with open(os.path.join(directory, filename), 'rb') as file:
            rawdata = file.read()
            result = chardet.detect(rawdata)
            encoding = result['encoding']
        if(encoding!="utf-8"):
            print(filename, encoding)


test#111#children#M.txt ascii
training#199#children#M.txt ascii
